In [1]:
import pandas as pd
import numpy as np
import keras
# from keras.preprocessing.image import load_img, img_to_array
from keras_preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [8]:
path= "corrected_csv_file.csv"
data = pd.read_csv(path)

image_folder = "capscum_org3/"

def load_data(dataframe, image_folder):
    images = []
    labels = []
    for _, row in dataframe.iterrows():
        img_path = image_folder + row["Image Name"]
        image = load_img(img_path, target_size=(224, 224))  # Resize as needed
        image = img_to_array(image)
        images.append(image)
        labels.append(row["SPAD Value"])
    return np.array(images), np.array(labels)


images, labels = load_data(data, image_folder)
# print(labels)
# Split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 0.5 of the remaining 0.3


In [9]:
print(len(y_test))
print(len(y_train))
print(len(y_val))
print(3600 / 141)

27
123
27
25.53191489361702


In [10]:
from keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator instances for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Use `flow` method to create data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
validation_generator = test_datagen.flow(X_val, y_val, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32)


In [12]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Define a simple regression model
model = Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error')



ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[373248,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [ ]:

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=1000
)

# Evaluate the model
test_loss = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')

In [ ]:
print(model.predict(X_test))
print(y_test[0])

1/1 [==============================] - 0s 73ms/step
[[11670.049 ]
 [10510.093 ]
 [11081.951 ]
 [11153.904 ]
 [10533.067 ]
 [10693.156 ]
 [11084.979 ]
 [10920.094 ]
 [12127.757 ]
 [10866.229 ]
 [11623.637 ]
 [ 8476.499 ]
 [11917.99  ]
 [10640.383 ]
 [11307.469 ]
 [11203.051 ]
 [ 7762.386 ]
 [ 7939.7246]
 [11393.443 ]
 [10073.033 ]
 [ 8607.476 ]
 [ 7872.5283]
 [ 7611.7817]
 [ 8123.459 ]
 [11207.424 ]
 [11216.839 ]
 [10627.206 ]]
92.1


[]
